In [1]:
from operator import itemgetter
from pathlib import Path
from typing import Iterator

import pandas as pd
import prodigy
import rapidfuzz
import spacy
from numpy import dot
from numpy.linalg import norm
from prodigy.components.filters import filter_duplicates
from prodigy.components.loaders import TXT
from prodigy.models.ner import EntityRecognizer
from prodigy.util import set_hashes
from spacy.kb import KnowledgeBase, Candidate  # , get_candidates

from gu_model.trf_tensor_to_vec import *

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
span = 'Trump'
person_of_interest='Donald Trump'
sentence="The man who would serve Trump as vice-president, Indiana governor Mike Pence, evaded any clear answer about his running mate’s position. In an interview aired Sunday with NBC’s Meet the Press, he said: “What Donald Trump laid out this week in Arizona was really a roadmap to end illegal immigration once and for all in this country.”"

In [3]:
kb_loc='assets/kb_full_2022_10_26'
nlp = spacy.load('gu_model/en_ner_guardian-1.0.3/en_ner_guardian/en_ner_guardian-1.0.3',
                     disable=['transformer', 'tagger', 'parser', 'lemmatizer', 'attribute_ruler'])
nlp.add_pipe('tensor2attr')
kb = KnowledgeBase(vocab=nlp.vocab, entity_vector_length=1)
kb.from_disk(kb_loc)
model = EntityRecognizer(nlp)

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_ner_guardian' (1.0.3) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/spacy_transformers/pipeline_component.py:406: UserWarning: Automatically converting a transformer component from spacy-transformers v1.0 to v1.1+. If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spacy-transformers version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [4]:
def get_candidates_from_fuzzy_matching(span, kb, matching_thres=60) -> Iterator[Candidate]:
    """
    Return a list of candidate entities for an alias based on fuzzy string matching.
    Each candidate defines the entity, the original alias,
    and the prior probability of that alias resolving to that entity.
    If the alias is not known in the KB, and empty list is returned.
    """
    aliases = kb.get_alias_strings()
    # matches=[]
    matches = {}
    for al in aliases:
        # fuzzy_ratio=rapidfuzz.fuzz.token_set_ratio(span.lower(),al.lower())
        fuzzy_ratio = rapidfuzz.fuzz.partial_ratio(span.lower(), al.lower())
        if fuzzy_ratio >= matching_thres:
            # matches.append(al)
            matches[al] = fuzzy_ratio
    candidates = []
    for match in matches:
        candidates.extend(kb.get_alias_candidates(match))
    return candidates, matches


def order_candidates_fuzzy_score(candidates, matches, candidate_limit=12):
    """
    Order candidates by descending fuzzy name matching score
    """
    # names = dict()
    candidate_d = dict()
    fuzzy_scores = dict()
    for candidate in candidates:
        qid = candidate.entity_
        name = candidate.alias_
        # names[qid] = name
        candidate_d[qid] = candidate
        fuzzy_scores[qid] = matches[name]
    entities_ordered = dict(sorted(fuzzy_scores.items(), key=itemgetter(1), reverse=True))
    entities_ordered = list(entities_ordered.keys())[:candidate_limit]
    return [candidate_d[entity] for entity in entities_ordered]

In [5]:
# Read the pre-defined CSV file into dictionaries mapping QIDs to the full names and descriptions
entity_loc = 'kb_datasets/kb_entities_full_2022_10_26.csv'
kb_entities = pd.read_csv(entity_loc, index_col=0)
kb_entities['id'] = kb_entities['id'].astype(str)
kb_entities['name'] = kb_entities['name'].astype(str)
kb_entities['desc'] = kb_entities['desc'].astype(str)
kb_entities['kb_url'] = kb_entities['kb_url'].astype(str)
kb_entities_url = kb_entities[['id', 'name', 'kb_url']]
kb_entities = kb_entities[['id', 'name', 'desc']]

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (2,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
kb_entities['desc_len']=kb_entities['desc'].str.len()

In [7]:
candidates, matches = get_candidates_from_fuzzy_matching(span, kb)

In [8]:
matches=pd.DataFrame.from_dict(matches, orient='index').reset_index().rename(columns={'index':'name',0:'score'})

In [9]:
matches=matches.merge(kb_entities, on=['name'], how='left')

In [10]:
score_thres=matches['score'].quantile(0.9)

In [11]:
matches=matches[matches['score']>=score_thres]

In [12]:
matches.shape

(1455, 5)

In [13]:
matches.sort_values(by='score', ascending=False).head(10)

,name,score,id,desc,desc_len
1916,Mary Macleod Trump,100.0,219217,This person is called Mary Macleod Trump. Mrs...,1253
7011,Maryanne Trump Barry,100.0,Q736223,Maryanne Trump Barry is a American lawyer and ...,460
6313,Melania Trump,100.0,116289,"Melania Trump is a former model, wife of Donal...",565
1273,Jules Trump,100.0,335179,Jules Trump is a Billionaire from Miami. Th...,124
5140,Charles S. Trump,100.0,Q27832616,Charles S. Trump is a American politician. Thi...,411
6456,Tiffany Trump,100.0,116275,Tiffany Trump is a Daughter of Donald Trump an...,734
1718,Fred C Trump,100.0,79677,Fred C Trump is a East coast real estate devel...,2739
4365,Fred C Trump Iii,100.0,219205,Fred C Trump III is a Nephew of Donald J. Trum...,94
1206,Stephanie Trump,100.0,176381,"Stephanie Trump is a Co-Founder, Williams Isla...",1420
242,Edmond Trump,100.0,242036,Edmond Trump is a Miami billionaire real estat...,1904


In [14]:
matches=matches.sort_values(by=['score','desc_len'], ascending=False).reset_index(drop=True)

In [15]:
matches.shape

(1455, 5)

In [16]:
matches.loc[matches['id'].str.match('Q\d'),'score']=matches.loc[matches['id'].str.match('Q\d'),'score'] * 1.05

In [17]:
max_len=300
min_len=matches['desc_len'].min()
matches['normalised_desc_len']=(matches['desc_len']-min_len)/(max_len-min_len)
matches['normalised_desc_len']=matches['normalised_desc_len'].apply(lambda x: 1 if x > 1 else x)

In [19]:
matches=matches.iloc[:200]

In [48]:
desc_ents_d = dict()
column_order=matches.reset_index().columns.values
id_loc,=np.where(column_order=='id')
desc_loc,=np.where(column_order=='desc')
for tuple_ in matches.itertuples():
    qid = tuple_[id_loc[0]]
    desc = tuple_[desc_loc[0]]
    desc_ents=nlp(desc).ents
    desc_ents_d[qid] = desc_ents

common_ent_count_d={}
sentence_ents = [str(ent) for ent in nlp(sentence).ents]
sentence_ents = set([ent for ent in sentence_ents if len(ent.split(' '))>1])
for qid, ent in desc_ents_d.items():
    ent=set([str(ent) for ent in ent])
    common_ent_counts=len(sentence_ents.intersection(ent))
    common_ent_count_d[qid]=common_ent_counts

In [49]:
common_ent_counts=pd.DataFrame.from_dict(common_ent_count_d,orient='index')\
    .reset_index().rename(columns={'index':'id',0:'common_ent_counts'})

In [50]:
matches=matches.merge(common_ent_counts, on='id')

In [51]:
matches=matches.sort_values(by=['common_ent_counts', 'score', 'normalised_desc_len'],ascending=False).reset_index(drop=True)

In [18]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [19]:
stopwords=set(stopwords.words())
def relevant_lexicon(sentence,stop_words=stopwords):
    punctuation_rm = str.maketrans('', '', string.punctuation)
    sentence = sentence.translate(punctuation_rm)
    sentence_tokens = set(word_tokenize(sentence))
    return sentence_tokens.difference(stopwords)

In [20]:
desc_ents_d = dict()
column_order=matches.reset_index().columns.values
id_loc,=np.where(column_order=='id')
desc_loc,=np.where(column_order=='desc')
for tuple_ in matches.itertuples():
    qid = tuple_[id_loc[0]]
    desc = tuple_[desc_loc[0]]
    desc_ents=relevant_lexicon(desc)
    desc_ents_d[qid] = desc_ents

In [21]:
common_ent_count_d={}
#sentence_ents = relevant_lexicon(sentence)

sentence_ents = [str(ent) for ent in nlp(sentence).ents]
sentence_ents =  set(' '.join(sentence_ents).split())

for qid, ent in desc_ents_d.items():
    common_ent_counts=len(sentence_ents.intersection(ent))
    common_ent_count_d[qid]=common_ent_counts

In [22]:
common_ent_counts=pd.DataFrame.from_dict(common_ent_count_d,orient='index')\
    .reset_index().rename(columns={'index':'id',0:'common_ent_counts'})

In [23]:
matches=matches.merge(common_ent_counts, on='id')

In [24]:
matches=matches.sort_values(by=['common_ent_counts', 'score', 'normalised_desc_len'],ascending=False).reset_index(drop=True)

In [25]:
matches.head(50)

,name,score,id,desc,desc_len,normalised_desc_len,common_ent_counts
0,Donald Trump,105.000000,Q22686,Donald Trump is a president of the United Stat...,476,1.000000,2
1,Maryanne Trump Barry,105.000000,Q736223,Maryanne Trump Barry is a American lawyer and ...,460,1.000000,2
2,Mary Anne Macleod Trump,105.000000,Q22952511,Mary Anne Macleod Trump is a mother of Donald ...,287,0.953237,2
3,Barron Trump,105.000000,Q23000814,Barron Trump is a son of Donald and Melania Tr...,274,0.906475,2
4,Donald Trump Jr.,105.000000,Q3713655,Donald Trump Jr. is a American businessman (bo...,234,0.762590,2
5,Elizabeth Trump Grau,105.000000,Q42682869,Elizabeth Trump Grau is a Donald Trump's siste...,218,0.705036,2
6,Fred C Trump,100.000000,79677,Fred C Trump is a East coast real estate devel...,2739,1.000000,2
7,Venessa Trump,100.000000,219251,Venessa Trump is a Estranged wife of Donald J ...,2013,1.000000,2
8,Edmond Trump,100.000000,242036,Edmond Trump is a Miami billionaire real estat...,1904,1.000000,2
9,Donald Trump Jr.,100.000000,102193,"Donald Trump Jr. is a EVP, The Trump Organizat...",1733,1.000000,2


In [ ]:
def order_candidates_fuzzy_score(candidates, matches, candidate_limit=12):
    """
    Order candidates by descending fuzzy name matching score
    """
    candidate_d = dict()
    fuzzy_scores = dict()
    for candidate in candidates:
        qid = candidate.entity_
        name = candidate.alias_
        candidate_d[qid] = candidate
        fuzzy_scores[qid] = matches[name]
    entities_ordered = dict(sorted(fuzzy_scores.items(), key=itemgetter(1), reverse=True))
    entities_ordered = list(entities_ordered.keys())[:candidate_limit]
    return [candidate_d[entity] for entity in entities_ordered]

In [38]:
matches['candidate']=matches['id'].map({candidate.entity_:candidate for candidate in candidates})

In [27]:
matches.head(10)

,name,score,id,desc,desc_len,normalised_desc_len,common_ent_counts
0,Donald Trump,105.0,Q22686,Donald Trump is a president of the United Stat...,476,1.000000,0
1,Maryanne Trump Barry,105.0,Q736223,Maryanne Trump Barry is a American lawyer and ...,460,1.000000,0
2,"Jean Barker, Baroness Trumpington",105.0,Q1297871,"Jean Barker, Baroness Trumpington is a British...",455,1.000000,0
3,Charles S. Trump,105.0,Q27832616,Charles S. Trump is a American politician. Thi...,411,1.000000,0
4,Lara Trump,105.0,Q35703322,Lara Trump is a American television producer a...,298,0.992806,0
5,Mary Anne Macleod Trump,105.0,Q22952511,Mary Anne Macleod Trump is a mother of Donald ...,287,0.953237,0
6,Barron Trump,105.0,Q23000814,Barron Trump is a son of Donald and Melania Tr...,274,0.906475,0
7,Mary L. Trump,105.0,Q96475328,Mary L. Trump is a American clinical psycholog...,243,0.794964,0
8,Tiffany Trump,105.0,Q12071552,Tiffany Trump is a American socialite (born 19...,237,0.773381,0
9,Donald Trump Jr.,105.0,Q3713655,Donald Trump Jr. is a American businessman (bo...,234,0.762590,0


In [28]:
matches[matches['name']==person_of_interest]

,name,score,id,desc,desc_len,normalised_desc_len,common_ent_counts
0,Donald Trump,105.0,Q22686,Donald Trump is a president of the United Stat...,476,1.0,0


In [ ]:
def embed_text(text, nlp):
    """
    Return spaCy embedding of a text.
    """
    return nlp(text).vector

In [ ]:
matches['desc_embedding']=matches['desc'].apply(lambda x: embed_text(x,nlp))

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [66]:
def calculate_cosine_similarity(descriptions_vec, vector_ref_sentence):
    """
    Return a dictionary mapping the kb entity id to cosine similarity score
    between kb embedded descriptions and the reference vector.
    """
    similarity = {}
    for entity_id in descriptions_vec.keys():
        vector_desc = descriptions_vec[entity_id]
        score = np.nan_to_num(
            dot(vector_ref_sentence, vector_desc) /
            (norm(vector_ref_sentence) * norm(vector_desc))
            , 0)
        similarity[entity_id] = score
    return similarity

In [67]:
def get_context_similarity(text, nlp, matches, candidate_limit=100):
    """
    Select only the top candidates to surface via the Prodigy UI. Based on
    topmost cosine similarities.
    """
    vector_ref_sentence = embed_text(text, nlp)
    
    names = dict()
    descriptions_vec = dict()
    column_order=matches.reset_index().columns.values
    id_loc,=np.where(column_order=='id')
    name_loc,=np.where(column_order=='name')
    desc_emb_loc,=np.where(column_order=='desc_embedding')
    for tuple_ in matches.itertuples():
        qid = tuple_[id_loc[0]]
        name = tuple_[name_loc[0]]
        desc_enc = tuple_[desc_emb_loc[0]]
        #desc_enc = candidate.entity_vector
        names[qid] = name
        descriptions_vec[qid] = desc_enc

    similarity = calculate_cosine_similarity(descriptions_vec, vector_ref_sentence)
    return pd.DataFrame.from_dict(similarity,orient='index')\
    .reset_index().rename(columns={'index':'id',0:'similarity'})

In [68]:
similarity_df=get_context_similarity(sentence, nlp,  matches)

In [69]:
similarity_df

,id,similarity
0,Q581095,0.955535
1,Q3195494,0.959758
2,Q6394699,0.955669
3,Q16187047,0.941213
4,Q6394636,0.933152
...,...,...
195,Q76229610,0.931908
196,Q75538116,0.949967
197,Q76101172,0.931816
198,Q76101163,0.932400


In [71]:
matches=matches.merge(similarity_df, on='id')

In [72]:
matches.head()

,name,score,id,desc,desc_len,normalised_desc_len,desc_embedding,similarity
0,Kerry Mccarthy,105.0,Q581095,Kerry Mccarthy is a British politician (born 1...,764,1.0,"[0.26950276, 0.2947241, -0.133328, 0.21762279,...",0.955535
1,Kerry-Lynne Findlay,105.0,Q3195494,Kerry-Lynne Findlay is a Canadian politician. ...,462,1.0,"[0.30053824, 0.26975125, -0.08049413, 0.258529...",0.959758
2,Kerryn Phelps,105.0,Q6394699,Kerryn Phelps is a Australian doctor and polit...,442,1.0,"[0.2647866, 0.30980346, -0.09035116, 0.2168619...",0.955669
3,Kerry Diotte,105.0,Q16187047,Kerry Diotte is a Canadian politician. This pe...,420,1.0,"[0.28495017, 0.36452746, -0.057482373, 0.23180...",0.941213
4,Kerry Roberts,105.0,Q6394636,Kerry Roberts is a Tennessee State Senator. Th...,398,1.0,"[0.27534282, 0.37202474, -0.11694094, 0.197256...",0.933152


In [75]:
matches.sort_values(by='similarity', ascending=False).head(10)

,name,score,id,desc,desc_len,normalised_desc_len,desc_embedding,similarity
47,Kerry Washington,100.0,250236,Kerry Washington is a Actress. Ms. Washington ...,771,1.0,"[0.34216702, 0.28275317, -0.2301654, 0.0876753...",0.981972
34,Kerry J Preete,100.0,60963,"Kerry J Preete is a EVP of Global Strategy, Mo...",1979,1.0,"[0.2719542, 0.2566482, -0.17158447, 0.05546593...",0.981368
41,Kerry Perry,100.0,279798,"Kerry Perry is a President and CEO, USA Gymnas...",1083,1.0,"[0.3057713, 0.27808952, -0.14171067, 0.1661069...",0.977658
35,Kerry Kennedy,100.0,38333,Kerry Kennedy is a Daughter of Robert F Kenned...,1733,1.0,"[0.28771013, 0.35336745, -0.14681476, 0.162945...",0.975714
45,Kerry Stokes,100.0,81059,Kerry Stokes is a Australian billionaire. Stro...,855,1.0,"[0.3257711, 0.25046766, -0.17361978, 0.1616998...",0.973949
36,Kerry Knott,100.0,21511,"Kerry Knott is a President, C.S. Lewis Institu...",1665,1.0,"[0.30162033, 0.30003712, -0.14818524, 0.136793...",0.973767
33,Kerry Halferty Hardy,100.0,143011,Kerry Halferty Hardy is a Principal and consul...,2529,1.0,"[0.3494941, 0.22872539, -0.18811136, 0.0616118...",0.972611
32,Kerry John Katsorhis,100.0,45631,"Kerry John Katsorhis is a Katsorhis Law Firm, ...",2896,1.0,"[0.31445464, 0.23945971, -0.12993431, 0.113871...",0.972049
39,R Kerry Clark,100.0,1324,R Kerry Clark is a Retired Chairman and Chief ...,1097,1.0,"[0.29704675, 0.30050045, -0.22320616, 0.144830...",0.970065
46,Kerry Mcconnon,100.0,330667,Kerry McConnon is a DDC Public Affairs. With D...,811,1.0,"[0.2904959, 0.3401988, -0.06810058, 0.11846647...",0.969262


In [73]:
matches[matches['name']==person_of_interest]

,name,score,id,desc,desc_len,normalised_desc_len,desc_embedding,similarity
37,John Kerry,100.0,13377,"John Kerry is a 68th US Secretary of State, Se...",1588,1.0,"[0.30509615, 0.22645071, -0.25140598, 0.139775...",0.956122


In [35]:
matches.sort_values(by=['similarity'], ascending=False).head(10)

,name,score,id,desc,desc_len,normalised_desc_len,desc_embedding,similarity
0,Carina Ståhl Herrstedt,105.0,Q4988712,Carina Ståhl Herrstedt is a Swedish politician...,589,1.0,"[0.2727859, 0.28228948, -0.10635452, 0.1914473...",0.035977
126,James Addison Halsted,100.0,175604,James Addison Halsted is a Nutrition researche...,1972,1.0,"[0.2816307, 0.27608034, -0.13001594, 0.1858776...",0.035977
128,Ted R French,100.0,3735,Ted R French is a Former Chief Financial Offic...,1739,1.0,"[0.30200538, 0.27971175, -0.18464883, 0.175322...",0.035977
129,William Bastedo,100.0,127217,"William Bastedo is a SVP, Booz Allen. As a Sen...",1728,1.0,"[0.2894372, 0.27440315, -0.16353847, 0.0848268...",0.035977
130,Ted Sarandos,100.0,128512,"Ted Sarandos is a Chief Content Officer, Netfl...",1702,1.0,"[0.29125437, 0.2391752, -0.12827414, 0.1505959...",0.035977
131,Shane Tedjarati,100.0,128384,"Shane Tedjarati is a President, Global High Gr...",1678,1.0,"[0.32086444, 0.21766777, -0.16003317, 0.140430...",0.035977
132,Ted Van Der Meid,100.0,26908,Ted Van Der Meid is counsel in our government...,1626,1.0,"[0.30236992, 0.28617683, -0.12945502, 0.149877...",0.035977
133,Shirley Mount Hufstedler,100.0,8420,Shirley Mount Hufstedler is a First US Secreta...,1622,1.0,"[0.33224115, 0.3046495, -0.14053863, 0.1369625...",0.035977
134,Ted W Love,100.0,65550,"Ted W Love is a EVP, Research & Development, O...",1573,1.0,"[0.26212552, 0.23311691, -0.19053678, 0.193209...",0.035977
135,Ted Malloch,100.0,255735,"Ted Malloch is a British Academic, Brexiteer, ...",1453,1.0,"[0.2903265, 0.28811792, -0.108777694, 0.115588...",0.035977


In [62]:
matches['composit_score']= matches['score'] * matches['similarity'] * matches['normalised_desc_len']

In [63]:
matches.sort_values('composit_score', ascending=False).head(20)

,name,id,desc,desc_len,desc_embedding,similarity,score,normalised_desc_len,composit_score
55,Imogene Powers Johnson,15168,"Imogene Powers Johnson, the billionaire widow...",2083,"[0.31178147, 0.3211938, -0.09199102, 0.1636720...",0.990551,90.0,1.00000,89.149590
58,Emory Johnson,164375,Emory Johnson is a Professor of Transportation...,2011,"[0.2981705, 0.23666798, -0.14443867, 0.0693922...",0.984195,90.0,1.00000,88.577587
34,Sue Clark-Johnson,178887,"Sue Clark-Johnson is a Former Publisher, Arizo...",2375,"[0.30413514, 0.3030189, -0.14652178, 0.1279459...",0.983990,90.0,1.00000,88.559128
22,Frederick Ross Johnson,149261,Frederick Ross Johnson is a CEO of RJR Nabisco...,2721,"[0.33529863, 0.3185673, -0.15305309, 0.0719855...",0.983683,90.0,1.00000,88.531485
23,Jennifer M Johnson,186127,"Jennifer M Johnson is a President and COO, Fra...",2713,"[0.33056182, 0.24376902, -0.14681502, 0.124611...",0.983338,90.0,1.00000,88.500436
7,Crawford Toy Johnson Iii,182518,Crawford Toy Johnson III is a Former Chairman ...,5272,"[0.31047255, 0.32346684, -0.1556873, 0.1069957...",0.982743,90.0,1.00000,88.446829
45,Mark Johnson,175844,"Mark Johnson is a COO, Civilian Marksmanship P...",2126,"[0.32258293, 0.26170307, -0.19030425, 0.044983...",0.980089,90.0,1.00000,88.207973
56,Ashland Johnson,390453,"Ashland Johnson is a Attorney, National Center...",2068,"[0.2945806, 0.22795784, -0.12959296, 0.0749545...",0.978972,90.0,1.00000,88.107525
15,Linda E Johnson,227813,"Linda E Johnson is a President and CEO, Brookl...",3519,"[0.31697896, 0.27082464, -0.17599843, 0.046157...",0.978441,90.0,1.00000,88.059728
21,E Christopher Johnson Jr,386811,E Christopher Johnson Jr is a CEO and Co-Found...,2941,"[0.330057, 0.21661668, -0.18811746, 0.07042361...",0.974086,90.0,1.00000,87.667750


In [108]:
matches.sort_values(by='score',ascending=False)

,name,score,id,desc,desc_len,desc_embedding,similarity
0,Jo Johnson,95.000000,Q728063,Jo Johnson is a British politician MP (born 19...,672,"[0.2822974, 0.24237156, -0.12491302, 0.2132171...",0.983038
2,Ed Johnson,95.000000,315140,This person is called Ed Johnson. This per...,80,"[0.3014737, -0.043383293, -0.17395419, -0.0418...",0.928145
3,Jo Johnson,95.000000,55090,Jo Johnson is a Prospective Parliamentary Cand...,66,"[0.38097247, -0.09801138, -0.19058442, 0.15013...",0.924945
4,Rj Johnson,95.000000,425078,RJ Johnson is a Top political advisor to Scott...,54,"[0.3856456, -0.16109328, -0.24016757, 0.184732...",0.902954
5,Ej Johnson,95.000000,331445,EJ Johnson is a Son of Magic Johnson.,37,"[0.39550623, -0.13490239, -0.26468384, 0.17075...",0.886878
...,...,...,...,...,...,...,...
2182,Johannes Poulsen,69.230769,Q12320116,Johannes Poulsen is a Danish politician. This ...,305,"[0.31842715, 0.2602151, -0.0374893, 0.24306317...",0.984012
2183,Johannes Gilleberg,69.230769,Q6216704,Johannes Gilleberg is a Norwegian politician. ...,304,"[0.30388454, 0.31487128, -0.093770236, 0.23683...",0.968860
2184,Sig Hutchinson,69.230769,315082,Sig Hutchinson is a Sig Hutchinson LLC/Wake Co...,301,"[0.24709877, 0.08559995, -0.19148602, 0.053567...",0.940625
2185,John Okechukwuemeka,69.230769,Q6251174,John Okechukwuemeka is a Nigerian politician. ...,298,"[0.32186395, 0.2897311, -0.0674257, 0.27010944...",0.979908


In [111]:
matches.sort_values(by='similarity',ascending=False).head(20)

,name,score,id,desc,desc_len,desc_embedding,similarity
29,Imogene Powers Johnson,90.000000,15168,"Imogene Powers Johnson, the billionaire widow...",2083,"[0.31178147, 0.3211938, -0.09199102, 0.1636720...",0.990551
1854,Johanna Quandt,69.230769,76711,Johanna Quandt is a Widow held 17% of BMW. For...,1684,"[0.28511798, 0.32589412, -0.1012195, 0.1773893...",0.989534
1884,John Sd Eisenhower,69.230769,50130,"John SD Eisenhower is a US Army General, Milit...",1178,"[0.27957487, 0.30162582, -0.10646973, 0.191410...",0.989325
86,Gary E Johnson,90.000000,34262,Gary E Johnson is a Former Governor of New Mex...,1203,"[0.32614115, 0.2979551, -0.15264152, 0.1700044...",0.989246
98,Clay A Johnson,90.000000,36869,Clay A Johnson is a Director of Sunlight Labs....,1105,"[0.31782562, 0.31476963, -0.08849746, 0.170402...",0.988494
1860,John Stanley Pottinger,69.230769,360836,"John Stanley Pottinger is a Attorney, novelist...",1528,"[0.2854411, 0.28591397, -0.12673561, 0.1575661...",0.987850
93,Ronald B Johnson,90.000000,2503,Ronald B Johnson is a Senior Vice President of...,1147,"[0.292732, 0.3121277, -0.15763398, 0.14204612,...",0.987792
65,Woody Johnson,90.000000,46087,Woody Johnson is a Great-grandson of Johnson &...,1452,"[0.28356323, 0.32903042, -0.08760893, 0.121867...",0.987749
155,Kendra R Johnson,90.000000,388307,This person is called Kendra R Johnson. Kendr...,703,"[0.30293167, 0.24478087, -0.16445631, 0.141636...",0.987638
1901,John Saunders,69.230769,182352,John Saunders is a ESPN Studio Host/Play-by-Pl...,1051,"[0.31606877, 0.32837492, -0.16691738, 0.130412...",0.987537


In [121]:
kb_entities[kb_entities['name']=='Boris Johnson']

,id,name,desc,desc_len
427360,Q180589,Boris Johnson,Boris Johnson is a Prime Minister of the Unite...,824


In [120]:
matches.sort_values(by=['score','similarity'], ascending=False).to_csv('similarity.csv')

In [25]:
kb_entities[kb_entities['name'].isin(candidates)]

,id,name,desc
236143,23759,Barry Strumpf,This person is called Barry Strumpf. This ...
245829,47307,Donald L Trump,"Donald L Trump is a President & CEO, Roswell P..."
266609,79677,Fred C Trump,Fred C Trump is a East coast real estate devel...
322132,176381,Stephanie Trump,"Stephanie Trump is a Co-Founder, Williams Isla..."
347611,219209,Mary Trump,"Mary Trump is a DJT niece, author of ""Too Much..."
347618,219217,Mary Macleod Trump,This person is called Mary Macleod Trump. Mrs...
356389,242036,Edmond Trump,Edmond Trump is a Miami billionaire real estat...
366407,260572,Robert Trumpbour,Robert Trumpbour is a Associate Professor of C...
392603,335179,Jules Trump,Jules Trump is a Billionaire from Miami. Th...
418528,408804,Ann Trump Daniel,Ann Trump Daniel is a Members of the UC Davis ...


In [3]:
##